In [71]:
import pandas as pd
import datetime
import plotly.express as px
import numpy as np
df = pd.read_pickle('health2.pickle')

In [73]:
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df

,date,bodymassindex_count_max,bodymassindex_count_min,bodymassindex_count_mean,bodymassindex_count_sum,bodymass_kg_max,bodymass_kg_min,bodymass_kg_mean,bodymass_kg_sum,bodyfatpercentage_%,...,running_steps_count_max,running_steps_count_min,running_steps_count_mean,running_steps_count_sum,multi_sport_distance_meters,multi_sport_duration_seconds,multi_sport_calories_cal,multi_sport_maxHR_bpm,multi_sport_averageHR_bpm,multi_sport_steps_count
date,,,,,,,,,,,,,,,,,,,,,
2015-11-30,2015-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-01,2015-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-02,2015-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-03,2015-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-04,2015-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-27,2022-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27,2022-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-28,2022-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# Add column for average speed on running
TRIP_DISTANCE_METERS = 6000
MARGIN_PERCANTAGE = 8
diff_meters = (MARGIN_PERCANTAGE / 100) * TRIP_DISTANCE_METERS
df['avg_speed_running_trip'] = df.apply(lambda row: (row.running_duration_seconds/60) / (row.running_distance_meters / 1000) if row.running_distance_meters and row.running_distance_meters - diff_meters <= row.running_distance_meters <= row.running_distance_meters + diff_meters else np.NaN, axis=1)
df


,date,bodymassindex_count_max,bodymassindex_count_min,bodymassindex_count_mean,bodymassindex_count_sum,bodymass_kg_max,bodymass_kg_min,bodymass_kg_mean,bodymass_kg_sum,bodyfatpercentage_%,...,running_steps_count_min,running_steps_count_mean,running_steps_count_sum,multi_sport_distance_meters,multi_sport_duration_seconds,multi_sport_calories_cal,multi_sport_maxHR_bpm,multi_sport_averageHR_bpm,multi_sport_steps_count,avg_speed_running_trip
date,,,,,,,,,,,,,,,,,,,,,
2015-11-30,2015-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-01,2015-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-02,2015-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-03,2015-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-04,2015-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-27,2022-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27,2022-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.381378
2022-03-28,2022-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
# combining where date occurs twice
relevant_columns = ('date', 'bodymass_kg_mean', 'stepcount_count_sum', 'distancewalkingrunning_km_sum', 'bodymass_kg', 'bloodpressuresystolic_mmHg', 'bloodpressurediastolic_mmHg', 'walking_distance_meters', 'running_distance_meters', 'avg_speed_running_trip')
strategy = {_: 'first' for _ in relevant_columns}
df = df.groupby(df.index).agg(strategy)
df.sort_index()
df.drop('date', axis=1, inplace=True)
df.tail(40)

,bodymass_kg_mean,stepcount_count_sum,distancewalkingrunning_km_sum,bodymass_kg,bloodpressuresystolic_mmHg,bloodpressurediastolic_mmHg,walking_distance_meters,running_distance_meters,avg_speed_running_trip
date,,,,,,,,,
2022-02-18,NaN,2463.0,1.528202,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-19,NaN,17739.0,11.845500,80.729,123.000,86.000,NaN,NaN,NaN
2022-02-20,NaN,7519.0,0.506180,NaN,NaN,NaN,765.239990,5052.509766,6.619915
2022-02-21,NaN,1785.0,7.369900,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-22,NaN,1474.0,0.899040,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-23,NaN,12166.0,7.340040,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-24,NaN,1761.0,1.078330,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-25,NaN,1431.0,0.881310,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-26,NaN,8302.0,1.346520,NaN,NaN,NaN,535.650024,5011.709961,6.326011


In [76]:
running_df = df.copy()
running_df = running_df.resample('M').agg({'avg_speed_running_trip':'max', 'running_distance_meters':'count'})
running_df.rename(columns={"running_distance_meters": "number_of_runs"}, inplace=True)
running_df.astype(float)
running_df['number_of_runs'].interpolate(method='linear', limit_direction='backward', axis=0, inplace=True)
running_df = running_df.ffill().bfill()
running_df

,avg_speed_running_trip,number_of_runs
date,,
2015-11-30,6.203283,0
2015-12-31,6.203283,0
2016-01-31,6.203283,0
2016-02-29,6.203283,0
2016-03-31,6.203283,0
...,...,...
2021-11-30,6.946182,3
2021-12-31,6.690722,2
2022-01-31,7.512827,2


In [77]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(x=running_df.index, y=running_df.number_of_runs, name="Number of runs", yaxis='y'))
fig.add_trace(go.Scatter(x=running_df.index, y=running_df.avg_speed_running_trip, name="Max minutes/Km", yaxis="y2"))

# Create axis objects
fig.update_layout(xaxis=dict(domain=[0.5, 0.5]),
	yaxis=dict(
		title="Number of runs",
		titlefont=dict(color="#1f77b4"),
		tickfont=dict(color="#1f77b4")),
	yaxis2=dict(title="Average speed", overlaying="y", side="right", position=1, autorange="reversed"))

fig.update_layout(title_text="Monthly run counts and max pace", width=900)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), legend_title_text=None)

fig.show()

In [78]:
df['running_distance_garmin'] = df.running_distance_meters.apply(lambda x: x / 1000 if not pd.isna(x) else 0)
df['walking_distance_garmin'] = df.walking_distance_meters.apply(lambda x: x / 1000 if not pd.isna(x) else 0)
df['walking_running_distance_applehealth'] = df.distancewalkingrunning_km_sum.apply(lambda x: x if not pd.isna(x) else 0)

df['walking_km'] = df.apply(lambda row: row.walking_running_distance_applehealth - row.running_distance_garmin if row.walking_running_distance_applehealth >= row.walking_distance_garmin + row.running_distance_garmin else row.walking_running_distance_applehealth, axis=1)
df['running_km'] = df.apply(lambda row: row.running_distance_garmin, axis=1)

df

,bodymass_kg_mean,stepcount_count_sum,distancewalkingrunning_km_sum,bodymass_kg,bloodpressuresystolic_mmHg,bloodpressurediastolic_mmHg,walking_distance_meters,running_distance_meters,avg_speed_running_trip,running_distance_garmin,walking_distance_garmin,walking_running_distance_applehealth,walking_km,running_km
date,,,,,,,,,,,,,,
2015-11-30,NaN,1405.0,0.934910,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,0.934910,0.934910,0.00000
2015-12-01,NaN,334.0,0.317780,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,0.317780,0.317780,0.00000
2015-12-02,NaN,2738.0,1.799660,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,1.799660,1.799660,0.00000
2015-12-03,NaN,6074.0,4.055108,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,4.055108,4.055108,0.00000
2015-12-04,NaN,6211.0,4.099860,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,4.099860,4.099860,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,NaN,8150.0,5.479471,NaN,124.666,81.666,NaN,NaN,NaN,0.00000,0.00000,5.479471,5.479471,0.00000
2022-03-26,NaN,8668.0,5.744744,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,5.744744,5.744744,0.00000
2022-03-27,NaN,14143.0,12.115723,NaN,NaN,NaN,NaN,8808.209961,6.381378,8.80821,0.00000,12.115723,3.307514,8.80821


In [79]:
df['total_distance_km'] = df.apply(lambda row: row.running_km + row.walking_km, axis=1)

In [80]:
# Fixing blood preassure
for _ in ('bloodpressurediastolic_mmHg', 'bloodpressuresystolic_mmHg', 'bodymass_kg'):
    df[_].replace(0.0, np.NaN, inplace=True)
    df[_].interpolate(method='linear', limit_direction='backward', inplace=True)

# filtering
df_all = df.copy()
f, t = datetime.date(2020, 1, 1), datetime.date(2023, 1, 1)
dff = df.query('date > @f & date < @t')
df_filtered = dff.copy()

In [81]:
dff = dff.resample('W').sum()
dff['walking_running_km_mean'] = dff.total_distance_km.mean()

dff

,bodymass_kg_mean,stepcount_count_sum,distancewalkingrunning_km_sum,bodymass_kg,bloodpressuresystolic_mmHg,bloodpressurediastolic_mmHg,walking_distance_meters,running_distance_meters,avg_speed_running_trip,running_distance_garmin,walking_distance_garmin,walking_running_distance_applehealth,walking_km,running_km,total_distance_km,walking_running_km_mean
date,,,,,,,,,,,,,,,,
2020-01-05,0.0,15344.0,9.351720,333.280333,500.096885,330.619513,0.000000,4022.239990,6.061270,4.02224,0.00000,9.351720,9.351720,4.02224,13.373960,25.041178
2020-01-12,0.0,15655.0,9.737703,584.640778,876.568135,580.147346,0.000000,11379.779785,13.129712,11.37978,0.00000,9.737703,9.737703,11.37978,21.117483,25.041178
2020-01-19,0.0,22414.0,15.072184,586.308593,865.813468,573.255319,0.000000,9369.219971,12.671419,9.36922,0.00000,15.072184,15.072184,9.36922,24.441404,25.041178
2020-01-26,0.0,49249.0,31.548989,587.976407,845.657787,559.702128,0.000000,0.000000,0.000000,0.00000,0.00000,31.548989,31.548989,0.00000,31.548989,25.041178
2020-02-02,0.0,17249.0,10.409071,589.644222,825.502106,546.148936,0.000000,11006.919922,12.363328,11.00692,0.00000,10.409071,10.409071,11.00692,21.415991,25.041178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-06,0.0,48561.0,35.076830,566.112143,876.952214,614.164857,0.000000,12042.820068,13.699147,12.04282,0.00000,35.076830,35.076830,12.04282,47.119650,25.041178
2022-03-13,0.0,71225.0,53.057274,567.285000,877.996500,614.664500,5015.460083,14217.189697,13.171935,14.21719,5.01546,53.057274,49.028094,14.21719,63.245283,25.041178
2022-03-20,0.0,62922.0,50.104996,480.989143,877.686786,611.711929,1597.640015,7016.919922,6.953840,7.01692,1.59764,50.104996,50.104996,7.01692,57.121916,25.041178


In [82]:
b = px.bar(dff, y=['walking_km', 'running_km'], title="Weekly distance", height=500)
b.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), legend_title_text=None)
l = px.line(dff, y="walking_running_km_mean")
b.add_traces(l.data)

In [83]:
dff = df_filtered.resample('W').max()
both = ["bloodpressuresystolic_mmHg", "bloodpressurediastolic_mmHg"]
for _ in both:
    dff[_] = dff[_].rolling(3).mean()
l = px.line(dff, y=both, title="Weekly blood pressure", height=500)
l.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), legend_title_text=None)

In [84]:
dff = df_filtered.resample('W').max()
dff['bodymass_kg'] = dff['bodymass_kg'].rolling(5).mean()
l = px.line(dff, y='bodymass_kg', title="Weekly average weight", height=500)
l.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01), legend_title_text=None)